In [1]:
#Load SentencePiece Model
import tensorflow as tf
import random
import glob
import os
from bert import tokenization_sentencepiece

model_file = 'model/sentencepiece/wiki-ko.model'
vocab_file = 'model/sentencepiece/wiki-ko.vocab'

tokenizer = tokenization_sentencepiece.FullTokenizer(
            model_file=model_file,
            vocab_file=vocab_file,
            do_lower_case=True)

max_seq_length = 256
#10% of sequences are shorter than max_seq_length
short_seq_prob = 0.1

# repeat 5 times
dupe_factor = 5
# Masks 15% of words
masked_lm_prob = 0.15 

# maximum_number of predictions per sequence
max_predictions_per_seq = 20

rng  = random.Random(12345)

vocab_words = list(tokenizer.vocab.keys())

instances = []

document_index = 0

Loaded a trained SentencePiece model.


# 0. Intro :  Bert Pre-Training Tasks

### Task No.1: Masked LM 
   * 15% of the words are masked at random
     and the task is to predict the masked words based on its
     left and right context
   * Not all tokens were masked in the same way
     (example sentence “My dog is hairy”)
   * 80% were replaced by the [MASK] token: “My dog is [MASK]”
   * 10% were replaced by a random token: “My dog is apple”
   * 10% were left intact: “My dog is hairy”
    
### Task No.2: Next Sentence Prediction
* Motivation
   * Many downstream tasks are based on understanding the relationship       between two text sentences
      * Question Answering (QA) and Natural Language Inference (NLI)
   * Language modeling does not directly capture that relationship
* The task is pre-training binarized next sentence prediction task

```
Input = [CLS] the man went to [MASK] store [SEP] he bought a gallon
[MASK] milk [SEP]
Label = isNext
Input = [CLS] the man [MASK] to the store [SEP]penguin [MASK] are
flight ##less birds[SEP]
Label = NotNext
```


# 1. Tokenization of Documents

  * We use sentencepiece to tokenize the documents
  * Refer to step2 for information on sentencepiece

### 1.1 tokenize using sentencepiece

In [3]:
input_files = ['data/wiki-kor/AA/test.txt']
all_documents = [[]]

# repeated dupe_fator(=5) times

for input_file in input_files:
    with tf.gfile.GFile(input_file, "r") as reader:
        while True:
            line = tokenization_sentencepiece.convert_to_unicode(reader.readline())
            if not line:
                break
            line = line.strip()

            # Empty lines are used as document delimiters
            if not line:
                all_documents.append([])
            tokens = tokenizer.tokenize(line)
            if tokens:
                all_documents[-1].append(tokens)

# Remove empty documents
all_documents = [x for x in all_documents if x]
#random.shuffle(all_documents)
print('Document 0 \n',all_documents[0])

Document 0 
 [['▁조봉암', '(', '曺', '奉', '岩', ',', '▁1898', '년', '▁9', '월', '▁25', '일', '▁~', '▁1959', '년', '▁7', '월', '▁31', '일', ')', '은', '▁대한민국의', '▁정치인이다', '.', '▁본관은', '▁창녕', '(', '昌', '寧', ')', '이다', '.', '▁사형', '집행', '될', '▁때', '▁유가족', '은', '▁친', '딸', '인', '▁호', '정', '(', '32', '세', ')', '씨와', '의', '▁아들인', '▁규', '호', '(11', '세', ',', '▁', '국민학교', '▁4', '학년', '▁재학', '),', '▁인천', '시', '에서', '▁거주하는', '▁친', '누', '이', '인', '▁조경', '암', '(', '66', '세', ')', '과', '▁그외', '▁조카', '▁등', '▁친척', '들이', '▁서울시', '와', '▁인천', '시', '에서', '▁살고', '▁있었다', '.'], ['▁그는', '▁일제', '▁강점기에', '▁소련', '으로', '▁건너가', '▁모스크바', '▁동방', '노', '력', '자', '공', '산', '대학', '을', '▁2', '년', '▁수료', '하고', ',', '▁1925', '년', '▁조선공산당', '이', '▁조직', '되었을', '▁때', ',', '▁조직', '중앙', '위원장', '을', '▁지냈으며', '▁고려', '공', '산', '청년회', '의', '▁간부', '가', '▁되었다', '.', '▁그해', '▁공산', '청년회', '▁대표로', '▁중국', '▁상하이', '(', '上', '海', ')', '를', '▁경유하여', '▁소련의', '▁모스크바에서', '▁열린', '▁코민테른', '▁총회', '에', '▁참석', '하고', ',', '▁모스크바', '▁동방', '노', '력', '자', '공', '산'

# 2. Run task No. 2 
  
   Joins sentences A and B together and forms the paragraph for next sentence prediction task. Sentence B can be next sentence or randomly selected paragraph from other document 

###  2.1 Define sentences with diffrent  target sequence length

  * Make some sequence shorter
     * 90% : `max_seq_length` 
     * 10% :  shorter sequences 

In [4]:
#Select a document

document = all_documents[1]

# Account for [CLS], [SEP], [SEP]
max_num_tokens = max_seq_length - 3

# Make some sequence shorter
#     90% : `max_seq_length` 
#     10% :  shorter sequences 

target_seq_length = max_num_tokens

if rng.random() < short_seq_prob:
    target_seq_length = rng.randint(2, max_num_tokens)

### 2.2 Task #2 

In [5]:
   
    # We split the input into segments "A" and "B" based on 
    # the actual "sentences" provided by the user input.

    instances = []
    current_chunk = []
    current_length = 0
    i = 0

    while i < len(document):
        print('current_iteration : ',i)
        segment = document[i]
        current_chunk.append(segment)
        current_length += len(segment)

        #  If the length of chunk is larger than target_seq_length
        if i == len(document) - 1 or current_length >= target_seq_length:
            if current_chunk:
                ### Token A
                
                # `a_end` :  how many segments from `current_chunk` go into the `A`
                a_end = 1
                if len(current_chunk) >= 2:
                    ##  Choose randomly the end of A sentence
                    a_end = rng.randint(1, len(current_chunk) - 1)
                    print('a_end : ',a_end)
                
                tokens_a = []
                for j in range(a_end):
                    tokens_a.extend(current_chunk[j])

                tokens_b = []
                # Random next
                is_random_next = False
                
                ### Token B
                
                # For 50% of cases, select paragraph from other document
                if len(current_chunk) == 1 or rng.random() < 0.5:
                    is_random_next = True
                    
                    # length of the token B
                    target_b_length = target_seq_length - len(tokens_a)

                    # Check random document is not same as the current one
                    for _ in range(10):
                        random_document_index = rng.randint(0, len(all_documents) - 1)
                        if random_document_index != document_index:
                            break
                            
                    
                    random_document = all_documents[random_document_index]
                    random_start = rng.randint(0, len(random_document) - 1)
                   
                    for j in range(random_start, len(random_document)):
                        tokens_b.extend(random_document[j])
                        if len(tokens_b) >= target_b_length:
                            break

                    # We used only a_end chunks
                    num_unused_segments = len(current_chunk) - a_end
                    i -= num_unused_segments
                    
                # For 50% of cases, select Next sentences
                else:
                    is_random_next = False
                    for j in range(a_end, len(current_chunk)):
                        tokens_b.extend(current_chunk[j])
 
                # truncates the sequence to a maximum length
                # randomly select A or B then remove tokens randomy from top ro bottom
                while True:
                    total_length = len(tokens_a) + len(tokens_b)
                    if total_length <= max_num_tokens:
                        break

                    # Truncate the longer sequence
                    trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
                    assert len(trunc_tokens) >= 1

                    # truncate randomly from the front and back 
                    if rng.random() < 0.5:
                        del trunc_tokens[0]
                    else:
                        trunc_tokens.pop()

                # Merge A and B into an Instance
                tokens = []
                segment_ids = []
                
                tokens.append("[CLS]")
                segment_ids.append(0)
                
                for token in tokens_a:
                    tokens.append(token)
                    segment_ids.append(0)

                tokens.append("[SEP]")
                segment_ids.append(0)

                for token in tokens_b:
                    tokens.append(token)
                    segment_ids.append(1)
                
                tokens.append("[SEP]")
                segment_ids.append(1)
                
                print(tokens,'\n',segment_ids)
                
                
                # Note : Task 2 Here....
           
            current_chunk = []
            current_length = 0
        i += 1


current_iteration :  0
['[CLS]', '▁토크', '는', '▁다음', '을', '▁가리킨다', '.', '[SEP]', '제', '▁경북', '중학교', '를', '▁졸업하고', '▁서울대학교', '▁법', '대에', '▁재학', '▁중이던', '▁1949', '년에', '▁제', '3', '회', '▁조선', '변호사', '시험', '에', '▁합격', '하였고', '▁1950', '년', '▁서울대학교', '를', '▁수석', '▁졸업했다', '.', '▁1951', '년부터', '▁1956', '년까지', '▁해군', '▁법무', '관으로', '▁군', '▁복무', '를', '▁마치고', '▁1957', '년', '▁대구', '지방법원', '▁판사', '에', '▁임용', '되었다', '.', '▁1961', '년부터', '▁1967', '년까지', '▁서울지방법원', '에서', '▁부장판사', ',', '▁1968', '년', '▁서울고등법원', '에서', '▁부장판사', '를', '▁하였다', '.', '▁1973', '년에', '▁임명된', '▁춘천', '지방법원', '장', '▁재직', '▁중이던', '▁1975', '년에', '▁일제시대', '▁법률', '교육을', '▁받지', '▁않은', '▁법조인', '으로서', '▁최초로', '▁대법원', '판사', '에', '▁임명되어', '▁10', '년간', '▁재직', '하였으며', '▁5', '공화국', '때', '▁법원', '행정', '처', '장을', '▁거쳐', '▁1986', '년', '▁4', '월', '▁16', '일에', '▁제', '9', '대', '▁대법원장', '에', '▁임명', '되었지만', '▁헌법', '개정', '으로', '▁5', '년', '▁임기', '를', '▁채', '우지', '▁못하고', '▁국회에서', '▁대법원장', '▁재임', '명', '▁여부', '를', '▁기다리', '다가', '▁1988', '년', '▁6', '월', '▁15',

# 3. Make Task No. 1 data

### 3.1   Run Task #1 create_masked_data

   - Randomly masks num_to_predict words
   - Put masks, or wrong answers or right answers


In [6]:
import collections
MaskedLmInstance = collections.namedtuple("MaskedLmInstance",
                                          ["index", "label"])

cand_indexes = []

# Do not mask of [CLS] or [SEP]
for (i, token) in enumerate(tokens):
    if token == "[CLS]" or token == "[SEP]":
        continue
    cand_indexes.append(i)

rng.shuffle(cand_indexes)

output_tokens = list(tokens)

# Select 15% of words to mask
num_to_predict = min(max_predictions_per_seq,
                     max(1, int(round(len(tokens) * masked_lm_prob))))

masked_lms = []
covered_indexes = set()
for index in cand_indexes:
    # Select 15% of words to mask
    if len(masked_lms) >= num_to_predict:
        break
    if index in covered_indexes:
        continue
    covered_indexes.add(index)

    masked_token = None
    # 80% of the time, replace with [MASK]
    if rng.random() < 0.8:
        masked_token = "[MASK]"
    else:
        # 10% of the time, keep original
        if rng.random() < 0.5:
            masked_token = tokens[index]
        # 10% of the time, replace with random word
        else:
            masked_token = vocab_words[rng.randint(0, len(vocab_words) - 1)]

    output_tokens[index] = masked_token

    masked_lms.append(MaskedLmInstance(index=index, label=tokens[index]))

masked_lms = sorted(masked_lms, key=lambda x: x.index)
print(masked_lms)

[MaskedLmInstance(index=20, label='▁제'), MaskedLmInstance(index=23, label='▁조선'), MaskedLmInstance(index=30, label='년'), MaskedLmInstance(index=34, label='▁졸업했다'), MaskedLmInstance(index=36, label='▁1951'), MaskedLmInstance(index=38, label='▁1956'), MaskedLmInstance(index=51, label='▁판사'), MaskedLmInstance(index=62, label='▁부장판사'), MaskedLmInstance(index=76, label='지방법원'), MaskedLmInstance(index=114, label='대'), MaskedLmInstance(index=163, label='▁되면서'), MaskedLmInstance(index=179, label='는'), MaskedLmInstance(index=185, label='▁길이'), MaskedLmInstance(index=191, label='며'), MaskedLmInstance(index=203, label='▁위한'), MaskedLmInstance(index=214, label='는'), MaskedLmInstance(index=221, label='▁계기가'), MaskedLmInstance(index=225, label='고'), MaskedLmInstance(index=231, label='장의'), MaskedLmInstance(index=251, label='▁것')]


### 3.2. Make it suitable for tensorflow dataset

In [7]:
masked_lm_positions = []
masked_lm_labels = []
for p in masked_lms:
    masked_lm_positions.append(p.index)
    masked_lm_labels.append(p.label)

print('masked_lm_positions \n',masked_lm_positions)
print('masked_lm_labels \n',masked_lm_labels)

masked_lm_positions 
 [20, 23, 30, 34, 36, 38, 51, 62, 76, 114, 163, 179, 185, 191, 203, 214, 221, 225, 231, 251]
masked_lm_labels 
 ['▁제', '▁조선', '년', '▁졸업했다', '▁1951', '▁1956', '▁판사', '▁부장판사', '지방법원', '대', '▁되면서', '는', '▁길이', '며', '▁위한', '는', '▁계기가', '고', '장의', '▁것']


# 4. Make Training Instance

### 4.1 Training Instance Class

In [8]:
class TrainingInstance(object):

    def __init__(self, tokens, segment_ids, masked_lm_positions,
                 masked_lm_labels,is_random_next):
        self.tokens = tokens
        self.segment_ids = segment_ids
        self.is_random_next = is_random_next
        self.masked_lm_positions = masked_lm_positions
        self.masked_lm_labels = masked_lm_labels

    def __str__(self):
        s = ""
        s += "tokens: %s\n" % (" ".join(
            [tokenization_sentencepiece.printable_text(x) for x in self.tokens]))
        s += "segment_ids: %s\n" % (" ".join([str(x) for x in self.segment_ids]))
        s += "is_random_next: %s\n" % self.is_random_next
        s += "masked_lm_positions: %s\n" % (" ".join(
            [str(x) for x in self.masked_lm_positions]))
        s += "masked_lm_labels: %s\n" % (" ".join(
            [tokenization_sentencepiece.printable_text(x) for x in self.masked_lm_labels]))
        s += "\n"
        return s

    def __repr__(self):
        return self.__str__()

### 4.2  Save each instance 

In [9]:
instance = TrainingInstance(
    tokens=tokens,
    segment_ids=segment_ids,
    is_random_next=is_random_next,
    masked_lm_positions=masked_lm_positions,
    masked_lm_labels=masked_lm_labels)
print(instance)

tokens: [CLS] ▁토크 는 ▁다음 을 ▁가리킨다 . [SEP] 제 ▁경북 중학교 를 ▁졸업하고 ▁서울대학교 ▁법 대에 ▁재학 ▁중이던 ▁1949 년에 ▁제 3 회 ▁조선 변호사 시험 에 ▁합격 하였고 ▁1950 년 ▁서울대학교 를 ▁수석 ▁졸업했다 . ▁1951 년부터 ▁1956 년까지 ▁해군 ▁법무 관으로 ▁군 ▁복무 를 ▁마치고 ▁1957 년 ▁대구 지방법원 ▁판사 에 ▁임용 되었다 . ▁1961 년부터 ▁1967 년까지 ▁서울지방법원 에서 ▁부장판사 , ▁1968 년 ▁서울고등법원 에서 ▁부장판사 를 ▁하였다 . ▁1973 년에 ▁임명된 ▁춘천 지방법원 장 ▁재직 ▁중이던 ▁1975 년에 ▁일제시대 ▁법률 교육을 ▁받지 ▁않은 ▁법조인 으로서 ▁최초로 ▁대법원 판사 에 ▁임명되어 ▁10 년간 ▁재직 하였으며 ▁5 공화국 때 ▁법원 행정 처 장을 ▁거쳐 ▁1986 년 ▁4 월 ▁16 일에 ▁제 9 대 ▁대법원장 에 ▁임명 되었지만 ▁헌법 개정 으로 ▁5 년 ▁임기 를 ▁채 우지 ▁못하고 ▁국회에서 ▁대법원장 ▁재임 명 ▁여부 를 ▁기다리 다가 ▁1988 년 ▁6 월 ▁15 일 ▁대법원장 ▁퇴진 을 ▁촉구 하는 ▁소장 ▁법관 들의 ▁서명 으로 ▁전개 된 ▁제 2 차 ▁사법 파 동에 ▁직면 하게 ▁되면서 ▁1988 년 ▁6 월 ▁17 일에 ▁대법원장 ▁접 견 실에서 ▁기자회견을 ▁갖고 ▁" 내가 ▁물러나 는 ▁것이 ▁사법부 가 ▁잘 되는 ▁길이 라고 ▁생각해 ▁물러난 다 " 며 ▁" 이 번 ▁서명 ▁사태 는 ▁소장 ▁법관 들의 ▁사법부 ▁발전을 ▁위한 ▁충 정 에서 ▁나온 ▁것으로 ▁보이기 에 ▁이들을 ▁비난 해서 는 ▁안 되며 ▁이를 ▁사법부 ▁발전 의 ▁계기가 ▁되어야 ▁한다 " 고 ▁하면서 ▁사 표를 ▁제출한 ▁법원 장의 ▁사 표를 ▁반 려 하고 ▁" 이 번 ▁사태 의 ▁모든 ▁책임 은 ▁사법부 ▁수장 인 ▁나 에게 ▁있는 ▁것 " 이라며 ▁1988 [SEP]
segment_ids: 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1

### 4.3 Write instance to .tfrecord


In [12]:
import collections

def create_int_feature(values):
  feature = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
  return feature


def create_float_feature(values):
  feature = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
  return feature

# Parameters

output_files = ['data/all.tfrecord']

instances.append(instance)

In [13]:
# Create TF example files from `TrainingInstance`s

writers = []

print('outter : ',type(output_files),output_files)


for output_file in output_files:
    writers.append(tf.python_io.TFRecordWriter(output_file))

writer_index = 0

total_written = 0


for (inst_index, instance) in enumerate(instances):
    input_ids = tokenizer.convert_tokens_to_ids(instance.tokens)
    input_mask = [1] * len(input_ids)
    segment_ids = list(instance.segment_ids)

    # Pad to maximum length
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)


    masked_lm_positions = list(instance.masked_lm_positions)
    masked_lm_ids = tokenizer.convert_tokens_to_ids(instance.masked_lm_labels)
    masked_lm_weights = [1.0] * len(masked_lm_ids)

    # pad to max_prediction_per_seq
    while len(masked_lm_positions) < max_predictions_per_seq:
        masked_lm_positions.append(0)
        masked_lm_ids.append(0)
        masked_lm_weights.append(0.0)

    next_sentence_label = 1 if instance.is_random_next else 0

    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(input_ids)
    features["input_mask"] = create_int_feature(input_mask)
    features["segment_ids"] = create_int_feature(segment_ids)
    features["masked_lm_positions"] = create_int_feature(masked_lm_positions)
    features["masked_lm_ids"] = create_int_feature(masked_lm_ids)
    features["masked_lm_weights"] = create_float_feature(masked_lm_weights)
    features["next_sentence_labels"] = create_int_feature([next_sentence_label])

    tf_example = tf.train.Example(features=tf.train.Features(feature=features))

    writers[writer_index].write(tf_example.SerializeToString())
    writer_index = (writer_index + 1) % len(writers)

    total_written += 1

    # Loggin

    if inst_index < 20:
        tf.logging.info("*** Example ***")
        tf.logging.info("tokens: %s" % " ".join(
          [tokenization_sentencepiece.printable_text(x) for x in instance.tokens]))

        for feature_name in features.keys():
            feature = features[feature_name]
            values = []
            if feature.int64_list.value:
                values = feature.int64_list.value
            elif feature.float_list.value:
                values = feature.float_list.value
            tf.logging.info(
                 "%s: %s" % (feature_name, " ".join([str(x) for x in values])))

for writer in writers:
    writer.close()

print("Wrote %d total instances", total_written)

outter :  <class 'list'> ['data/all.tfrecord']
INFO:tensorflow:*** Example ***
INFO:tensorflow:tokens: [CLS] ▁토크 는 ▁다음 을 ▁가리킨다 . [SEP] 제 ▁경북 중학교 를 ▁졸업하고 ▁서울대학교 ▁법 대에 ▁재학 ▁중이던 ▁1949 년에 ▁제 3 회 ▁조선 변호사 시험 에 ▁합격 하였고 ▁1950 년 ▁서울대학교 를 ▁수석 ▁졸업했다 . ▁1951 년부터 ▁1956 년까지 ▁해군 ▁법무 관으로 ▁군 ▁복무 를 ▁마치고 ▁1957 년 ▁대구 지방법원 ▁판사 에 ▁임용 되었다 . ▁1961 년부터 ▁1967 년까지 ▁서울지방법원 에서 ▁부장판사 , ▁1968 년 ▁서울고등법원 에서 ▁부장판사 를 ▁하였다 . ▁1973 년에 ▁임명된 ▁춘천 지방법원 장 ▁재직 ▁중이던 ▁1975 년에 ▁일제시대 ▁법률 교육을 ▁받지 ▁않은 ▁법조인 으로서 ▁최초로 ▁대법원 판사 에 ▁임명되어 ▁10 년간 ▁재직 하였으며 ▁5 공화국 때 ▁법원 행정 처 장을 ▁거쳐 ▁1986 년 ▁4 월 ▁16 일에 ▁제 9 대 ▁대법원장 에 ▁임명 되었지만 ▁헌법 개정 으로 ▁5 년 ▁임기 를 ▁채 우지 ▁못하고 ▁국회에서 ▁대법원장 ▁재임 명 ▁여부 를 ▁기다리 다가 ▁1988 년 ▁6 월 ▁15 일 ▁대법원장 ▁퇴진 을 ▁촉구 하는 ▁소장 ▁법관 들의 ▁서명 으로 ▁전개 된 ▁제 2 차 ▁사법 파 동에 ▁직면 하게 ▁되면서 ▁1988 년 ▁6 월 ▁17 일에 ▁대법원장 ▁접 견 실에서 ▁기자회견을 ▁갖고 ▁" 내가 ▁물러나 는 ▁것이 ▁사법부 가 ▁잘 되는 ▁길이 라고 ▁생각해 ▁물러난 다 " 며 ▁" 이 번 ▁서명 ▁사태 는 ▁소장 ▁법관 들의 ▁사법부 ▁발전을 ▁위한 ▁충 정 에서 ▁나온 ▁것으로 ▁보이기 에 ▁이들을 ▁비난 해서 는 ▁안 되며 ▁이를 ▁사법부 ▁발전 의 ▁계기가 ▁되어야 ▁한다 " 고 ▁하면서 ▁사 표를 ▁제출한 ▁법원 장의 ▁사 표를 ▁반 려 하고 ▁" 이 번 ▁사태 의 ▁모든 ▁